In [ ]:
#code of Interactive Quantum Classifier Inspired by Quantum Open System Theory
#LINK https://ieeexplore.ieee.org/document/9533917

#LINK https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9533917

#this code was written by Fernando Maciano de Paula Neto (fernando@cin.ufpe.br)

In [ ]:
import numpy as np
from scipy.linalg import expm as expMatrix
# Importing the necessary Python libraries
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score # Getting the Iris dataset from Scikit-Learn
from sklearn import preprocessing

def create_sigmaE(vectorX, vectorW):
  #x = x1,x2,x2,...,xn
  n = len(vectorX)
  sigmaE = np.zeros((n,n))
  for i in range(n):
    #print("VETOR X", vectorX)
    sigmaE[i,i] = vectorX[i]*vectorW[i]
  
  return sigmaE

def create_sigmaQ(n):
  sigmaQ = np.zeros((n,n))
  sigmaX = np.array([[0,1], [1,0]])
  sigmaY = np.array([[0,-1j], [1j,0]])
  sigmaZ = np.array([[1,0], [0,-1]])
  sigmaQ = sigmaX + sigmaY + sigmaZ

  return sigmaQ

def create_Uoperator(sigmaQ, sigmaE):
  return np.matrix(expMatrix(1j*np.kron(sigmaQ, sigmaE)))

def create_p(psi):
  #psi_cog is a quantum column vector in the form [[alfa],[beta]]
  psi = np.matrix(psi)
  return psi * psi.getH()


def create_classifier(vectorX, vectorW):
  sigmaQ = create_sigmaQ(2)
  sigmaE = create_sigmaE(vectorX, vectorW)
  U_operator = create_Uoperator(sigmaQ, sigmaE)
  p_cog = create_p([[1/np.sqrt(2)],[1/np.sqrt(2)]]) #|psi> = 1/sqrt(2)|0> + 1/sqrt(2)|1>
  N = len(vectorX)
  p_env = create_p([[1/np.sqrt(N)] for i in range(N)]) #|psi_env> = 1/sqrt(N)|0> + 1/sqrt(N)|1> + ... + 1/sqrt(N)|N-1>

  quantum_operation = np.array(U_operator * (np.kron(p_cog, p_env)) * U_operator.getH())
  #np.trace(A.reshape(n,m,n,m), axis1=0, axis2=2), and the trace with respect to the second subsystem (of dimension m) is np.trace(A.reshape(n,m,n,m), axis1=1, axis2=3)
  #n is the input dimension and m is the env dimension
  p_cog_new = np.trace(quantum_operation.reshape([2,N,2,N]), axis1=1, axis2=3)

  p_cog_new_00_2 = p_cog_new[0,0]
  p_cog_new_11_2 = p_cog_new[1,1]
  #print("matrix p", p_cog_new)
  if (p_cog_new_00_2 >= p_cog_new_11_2):
    z = 0
  else: #elif (p_cog_new_00_2 < p_cog_new_11_2):
    z = 1
  #print(abs(p_cog_new_00_2), abs(p_cog_new_11_2))
  return z, p_cog_new_11_2

def adjust_weights(w, y,z,x, p, n):
  w = w-n*(z-y)*(1-p)*x
  return w

def create_many_classifiers(vectorX, vectorWs):
  list_zi = []
  list_p11_i = []

  bigger_p11 = 0
  index_of_bigger = 0
  for vectorWs_i in vectorWs:
    zi, p11_i = create_classifier(vectorX, vectorWs_i)  
    list_zi.append(zi)
    list_p11_i.append(p11_i)
  
  return np.argmax(list_p11_i)

def training_1step_classifier(X, Y, w, n=0.1):
  error = 0
  for x,y in zip(X,Y):
    z, p11 = create_classifier(x,w)
    w = adjust_weights(w,y,z,x,p11,n)
    if (z != y):
      error += 1
  return w, error

def training_1batch_classifier(X, Y, w, n=0.1):
  error = 0
  x_avg = 0
  y_avg = 0
  z_avg = 0
  p11_avg = 0
  lines = X.shape[0]

  for x,y in zip(X,Y):
    z, p11 = create_classifier(x,w)
    x_avg += x/lines
    y_avg += y/lines
    z_avg += z/lines
    p11_avg += p11/lines
    if (z != y):
      error += 1
  w = adjust_weights(w,y_avg,z_avg,x_avg,p11_avg,n)

  return w, error

import math
def training_nsteps_classifier(X, Y, w, batch_size, Nsteps, n=0.1, stop_error_zero=False):
  lines = X.shape[0]
  divisions = math.ceil(lines / batch_size)
  
  min_error =1000000
  min_w_error = 100000
  for i in range(Nsteps):
    errors = 0
    for division in range(divisions):
      X_batch = X[division*batch_size:(division+1)*batch_size , :]
      Y_batch = Y[division*batch_size:(division+1)*batch_size]
      w_old = w[:]
      w, error = training_1batch_classifier(X_batch,Y_batch,w=w,n=n)
      errors += error
    if min_error > errors:
      min_w_error = w_old
      min_error = errors
      
    if errors==0 and stop_error_zero:
      break
  return min_w_error, min_error

def replicate_classes(data_set, features_names, classes):

  iris = data_set #datasets.load_iris()# Loading the predictor value (y) and remainder of the training dataset (X) as Pandas DataFrames

  list_x_y = []
  n_classes = len(classes)
  #iris = datasets.load_iris()
  for class_i in classes:
    y_class_i = data_set.copy() #pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
    #CLASSE 0 (0,1,2, ..., n-1) E CLASSE 1 (i)
    y_class_i.loc[ y_class_i["target"] == class_i  , "target"]  = n_classes+1
    y_class_i.loc[ y_class_i["target"] < n_classes  , "target"]  = 0
    y_class_i.loc[ y_class_i["target"] == (n_classes+1)  , "target"]  = 1
    y_class_i = pd.concat([y_class_i, y_class_i[y_class_i["target"]==1]], axis=0)
  
    list_x_y.append([y_class_i[features_names], y_class_i["target"]])

  return list_x_y
  
def training_k_fold_classifier(kfold, X, y, Nsteps, batch_size, n_learning_rate, features_names, classes, random_state=42):
    # Instantiating the K-Fold cross validation object with 5 folds
    k_folds = StratifiedKFold(n_splits = kfold, shuffle = True, random_state = random_state)# Iterating through each of the folds in K-Fold
    metrics = []
    for train_index, val_index in k_folds.split(X, y):
      print("TRAIN INDEX", train_index, "VAL_INDEX", val_index)
      # Splitting the training set from the validation set for this specific fold
      X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
      y_train, y_val = y.iloc[train_index], y.iloc[val_index]
      
      dataset = pd.concat([X_train, y_train], axis=1)
      #print("DATASET", dataset)
      #X1, y1, X2, y2, X3, y3 
      list_of_x_y = replicate_classes(dataset, features_names, classes)
      list_of_trainned_w = []
      count=0
      for Xi, yi in list_of_x_y:
        w = [0.1 for i in range(Xi.shape[1])]
        X_train = np.array(Xi)
        y_train = np.array(yi)
        wi, error = training_nsteps_classifier(X_train,y_train, w, Nsteps=Nsteps, batch_size=batch_size, n=n_learning_rate, stop_error_zero=True)
        print("error i", count, error)
        list_of_trainned_w.append(wi)
        count+=1

      #TESTING
      precision, recall, f1_measure, accuracy = test_many_classifiers(X_val , y_val, list_of_trainned_w )

      metrics.append([precision, recall, f1_measure, accuracy])
      print("metricas, precision, recall, f1_measure, acc", metrics)
    return list_of_trainned_w, metrics


def test_many_classifiers(X, y, list_of_weights_classifiers):
  lines = X.shape[0]
  y_pred = []
  for i in range(lines):
    output_class = create_many_classifiers(X.iloc[i, :].values, list_of_weights_classifiers)
    #print(X.iloc[i, :].values, output_class, y.iloc[i, :].values)
    y_pred.append(output_class)
  
  y_true = y["target"].tolist()

  hit=0
  for y, yhat in zip(y_true, y_pred):
    if (y == yhat):
      hit+=1
  print("ACERTOS",hit)
  precision = precision_score(y_true, y_pred, labels=[0,1,2], average='micro') #y_true, y_pred, labels=[1,2], average='micro'
  recall = recall_score(y_true, y_pred, labels=[0,1,2],average='micro')
  f1_measure = f1_score(y_true, y_pred, labels=[0,1,2], average='micro')
  accuracy = accuracy_score(y_true, y_pred)
  print("precision", precision, "recall", recall, "f1", f1_measure)
  print("Y TRUE",y_true, "Y_PRED",y_pred)
  return precision, recall, f1_measure, accuracy

def avg_metrics(measures):
  precisions=0
  recalls=0
  f1_measures=0
  accs=0
  count=0
  for precision, recall, f1_measure, accuracy in measures:
    precisions += precision
    recalls += recall
    f1_measures += f1_measure
    accs += accuracy
    count+=1
  print("acc", accs/count)
  print("precision", precisions/count)
  print("recall", recalls/count)
  print("f1_measure", f1_measures/count)

def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled

# apply the min-max scaling in Pandas using the .min() and .max() methods
def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm

def min_max_scaling_type_2(df):
    # copy the dataframe
    a, b= -1, 0
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        df_norm[column] =(b-a)*(df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())+ a
        
    return df_norm

In [ ]:
#como eh a normalizacao? eh por coluna ou por linha?
# tem batch e a atualizacao dos pesos acontece com a media dos valores envolvidos (media de x, media de p00, etc)
#
iris = datasets.load_iris()

X = iris.data
# normalize the data attributes
iris['data'] = -1*preprocessing.normalize(X)

X_iris = pd.DataFrame(data= iris['data'],
                      columns= iris['feature_names'])
y_iris = pd.DataFrame(data= iris['target'],
                      columns= ['target'])

#X_iris  = min_max_scaling_type_2(X_iris)

list_of_trainned_w, metrics = training_k_fold_classifier(kfold=10, X=X_iris, y=y_iris, Nsteps=2000, batch_size=1, n_learning_rate=0.009,features_names =iris['feature_names'], classes=[0,1,2],random_state=1) 

TRAIN INDEX [  0   1   2   3   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  26  27  28  29  30  31  32  33  34  35  36  37  38
  40  42  43  44  45  46  47  48  49  50  51  52  53  54  55  57  58  59
  60  61  62  63  64  65  66  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  84  85  86  88  89  90  91  92  93  95  96  97  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 114 115 116 118 119
 120 121 122 123 124 125 126 128 129 130 131 133 134 135 136 137 138 139
 140 141 142 143 144 146 147 148 149] VAL_INDEX [  4   5  25  39  41  56  67  87  94  98 113 117 127 132 145]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


error i 0 0
error i 1 5
error i 2 3
ACERTOS 14
precision 0.9333333333333333 recall 0.9333333333333333 f1 0.9333333333333333
Y TRUE [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2] Y_PRED [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
metricas, precision, recall, f1_measure, acc [[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]]
TRAIN INDEX [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  38  39  40  41  42  44  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  73  74  75  76  78
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  98
 100 101 102 103 105 106 107 108 109 110 111 112 113 114 115 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 139 141 142 143 144 145 146 147 149] VAL_INDEX [ 36  37  43  45  46  72  77  79  97  99 104 116 138 140 148]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


error i 0 0
error i 1 4
error i 2 3
ACERTOS 15
precision 1.0 recall 1.0 f1 1.0
Y TRUE [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2] Y_PRED [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
metricas, precision, recall, f1_measure, acc [[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0]]
TRAIN INDEX [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  20  21  22  23  24  25  26  28  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  45  46  48  49  50  51  52  55  56  57  58  59  60
  61  63  64  65  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 112 113 114 115 116 117 118
 119 120 121 122 123 124 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 142 143 145 146 147 148] VAL_INDEX [ 19  27  29  44  47  53  54  62  66  92 111 125 141 144 149]


...

error i 0 0
error i 1 4
error i 2 3
ACERTOS 15
precision 1.0 recall 1.0 f1 1.0
Y TRUE [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2] Y_PRED [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
metricas, precision, recall, f1_measure, acc [[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0]]
TRAIN INDEX [  0   1   3   4   5   6   7   8   9  10  11  12  13  14  16  17  18  19
  21  22  23  24  25  26  27  28  29  30  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  49  50  51  53  54  55  56  58  59  61
  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79
  80  82  83  84  85  87  88  89  90  91  92  93 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


error i 0 0
error i 1 5
error i 2 4
ACERTOS 14
precision 0.9333333333333333 recall 0.9333333333333333 f1 0.9333333333333333
Y TRUE [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2] Y_PRED [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2]
metricas, precision, recall, f1_measure, acc [[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]]
TRAIN INDEX [  0   1   2   3   4   5   7   9  10  11  12  13  14  15  17  18  19  20
  21  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


error i 0 0
error i 1 4
error i 2 3
ACERTOS 14
precision 0.9333333333333333 recall 0.9333333333333333 f1 0.9333333333333333
Y TRUE [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2] Y_PRED [0, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2]
metricas, precision, recall, f1_measure, acc [[0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [0.8666666666666667, 0.8666666666666667, 0.8666666666666667, 0.8666666666666667], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [1.0, 1.0, 1.0, 1.0], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333], [0.9333333333333333, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]]


In [ ]:
measures = metrics

avg_metrics(measures)

acc 0.8133333333333332
precision 0.8133333333333332
recall 0.8133333333333332
f1_measure 0.8133333333333332


In [ ]:
iris = datasets.load_iris()

X = iris.data
# normalize the data attributes
#iris['data'] = -1*min_max_scaling(pd.DataFrame(X))#-1*preprocessing.normalize(X)

X_iris = pd.DataFrame(data= iris['data'],
                      columns= iris['feature_names'])
y_iris = pd.DataFrame(data= iris['target'],
                      columns= ['target'])

X_iris  = min_max_scaling_type_2(X_iris)
#X_iris = pd.Pandas(iris['data'])
#y_iris = iris['target']

list_of_trainned_w, metrics = training_k_fold_classifier(kfold=10, X=X_iris, y=y_iris, Nsteps=2000, batch_size=1, n_learning_rate=0.009,features_names =iris['feature_names'], classes=[0,1,2],random_state=1) 

In [ ]:
measures = metrics

avg_metrics(measures)

In [ ]:
X = np.array([[-0.5, -0.7, -0.9, -1], [-0.0, -0.2, -0.1, -0.2]])
Y = np.array([[0],[1]])
w = [-0.1 for i in range(len(X[0]))]
w, error = training_nsteps_classifier(X,Y, w, Nsteps=100, n=0.01, stop_error_zero=False)
print(w, error)

In [ ]:
db1, db2, db3 = replicate_classes()

In [ ]:
iris = datasets.load_iris()
X = db1[iris['feature_names']]
y = db1['target']

w, errors = training_k_fold_classifier(kfold=2, X=X, y=y, random_state=42)

In [ ]:
#y_class_1.value_counts()
#y_class_1[y_class_1["target"]==0]
#pd.DataFrame(data = iris['data'], columns = ['feature_names', 'target']).copy()
#iris = datasets.load_iris()# Loading the predictor value (y) and remainder of the training dataset (X) as Pandas DataFrames

#data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     #columns= iris['feature_names'] + ['target'])#
#data1

In [ ]:
#X = pd.DataFrame(data = iris['data'], columns = iris['feature_names'])
#X = pd.DataFrame(-1*preprocessing.normalize(X))

In [ ]:
#X

In [ ]:
#y_class_2["target"].value_counts()


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

cm = confusion_matrix([0,1,2], [0,1,2])

In [ ]:
cm

In [ ]:
recall